# 数据的存储
1. 从TXT中加载数据，向量化之后存储到Chroma向量数据库

In [1]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import os
import dotenv
from oauthlib.uri_validate import query

dotenv.load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# 1. 加载
loader = TextLoader(
    file_path="./assets/ai.txt",
    encoding="utf-8",
)

docs = loader.load()

# 2. 创建文本拆分器
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

splitter_docs = text_splitter.split_documents(docs)

# 3. 创建嵌入模型
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# 4. 文档以及嵌入模型传入到Chroma相关的结构中，存储
db = Chroma.from_documents(
    documents=splitter_docs,
    embedding=embedding_model
)



问题一：当前的数据存在哪里了？

如果使用的from_documents方法没有显式的指明存储位置的话，将当前的数据存储到内存中，并缓存起来。
如果需要指定具体的存储位置，需要设置参数:persist_directory的值。

In [2]:
db1 = Chroma.from_documents(
    documents=splitter_docs,
    embedding=embedding_model,
    persist_directory="./assets/chroma-1"
)

问题二：在向量数据库中，不仅存储了数据（文档）的向量，还存储了数据本身。

演示一下检索的需求

In [3]:
query = "AI的挑战有哪些呢？"

docs = db.similarity_search(query)

print(docs[0].page_content)

------------------------------------------------------------
五、AI 的应用领域
医疗：智能诊断、药物研发
金融：智能风控、投顾系统
教育：个性化学习、智能批改
制造：质量检测、自动化生产
零售：推荐系统、智能客服
内容创作：文本生成、图像生成、音乐创作

------------------------------------------------------------
六、AI 的工作流程
1. 收集数据
2. 清洗数据
3. 训练模型
4. 评估模型
5. 部署应用

------------------------------------------------------------
七、AI 的挑战
- 数据隐私与安全
- 偏见与歧视问题
- 黑箱问题（模型不可解释）
- 职业替代与社会影响
- 伦理与法律风险

------------------------------------------------------------
八、AI 的未来趋势
1. 多模态AI（文字+图片+语音融合）
2. 通用人工智能（AGI）
3. 边缘AI（本地智能处理）
4. AI与各行业深度融合
5. AI治理与伦理框架

------------------------------------------------------------
九、总结
AI 是让机器具备“思考与学习能力”的技术体系，
它不仅是一门科学，更是一种推动社会变革的核心力量。


In [4]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

raw_documents = [
    Document(
        page_content="葡萄是一种常见的水果，属于葡萄科葡萄属植物。它的果实呈圆形或椭圆形，颜色有绿色、紫色、红色等多种。葡萄富含维生素C和抗氧化物质，可以直接食用或酿造成葡萄酒。",
        metadata={"source": "水果", "type": "植物"}
    ),
    Document(
        page_content="白菜是十字花科蔬菜，原产于中国北方。它的叶片层层包裹形成紧密的球状，口感清脆微甜。白菜富含膳食纤维和维生素K，常用于制作泡菜、炒菜或煮汤。",
        metadata={"source": "蔬菜", "type": "植物"}
    ),
    Document(
        page_content="狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。",
        metadata={"source": "动物", "type": "哺乳动物"}
    ),
    Document(
        page_content="猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。",
        metadata={"source": "动物", "type": "哺乳动物"}
    ),
    Document(
        page_content="人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。",
        metadata={"source": "生物", "type": "灵长类"}
    ),
    Document(
        page_content="太阳是太阳系的中心恒星，直径约139万公里，主要由氢和氦组成。它通过核聚变反应产生能量，为地球提供光和热。太阳活动周期约为11年，会影响地球气候。",
        metadata={"source": "天文", "type": "恒星"}
    ),
    Document(
        page_content="长城是中国古代的军事防御工程，总长度超过2万公里。它始建于春秋战国时期，秦朝连接各段，明朝大规模重修。长城是世界文化遗产和人类建筑奇迹。",
        metadata={"source": "历史", "type": "建筑"}
    ),
    Document(
        page_content="量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。",
        metadata={"source": "物理", "type": "科学"}
    ),
    Document(
        page_content="《红楼梦》是中国古典文学四大名著之一，作者曹雪芹。小说以贾、史、王、薛四大家族的兴衰为背景，描绘了贾宝玉与林黛玉的爱情悲剧，反映了封建社会的种种矛盾。",
        metadata={"source": "文学", "type": "小说"}
    ),
    Document(
        page_content="新冠病毒（SARS-CoV-2）是一种可引起呼吸道疾病的冠状病毒。它通过飞沫传播，主要症状包括发热、咳嗽、乏力。疫苗和戴口罩是有效的预防措施。",
        metadata={"source": "医学", "type": "病毒"}
    )
]

embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

db = Chroma.from_documents(
    documents=raw_documents,
    embedding=embedding,
    persist_directory="./assets/chroma-2"
)

In [5]:
query = "哺乳动物"

docs = db.similarity_search(query, k=3)
print(f"查询：{query}的结果")
for i, doc in enumerate(docs):
    print(f"\n结果 {i}")
    print(f"内容：{doc.page_content}")
    print(f"元数据：{doc.metadata}")

查询：哺乳动物的结果

结果 0
内容：猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。
元数据：{'source': '动物', 'type': '哺乳动物'}

结果 1
内容：狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。
元数据：{'type': '哺乳动物', 'source': '动物'}

结果 2
内容：人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。
元数据：{'source': '生物', 'type': '灵长类'}


In [6]:
query = "哺乳动物"
embedding_vector = embedding.embed_query(query)

docs = db.similarity_search_by_vector(embedding_vector, k = 3)

print(f"查询：'{query}' 的结果：")
for i, doc in enumerate(docs):
    print(f"\n结果 {i}")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")

查询：'哺乳动物' 的结果：

结果 0
内容: 猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。
元数据: {'type': '哺乳动物', 'source': '动物'}

结果 1
内容: 狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。
元数据: {'type': '哺乳动物', 'source': '动物'}

结果 2
内容: 人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。
元数据: {'source': '生物', 'type': '灵长类'}


In [9]:
query = "哺乳动物"

docs = db.similarity_search(
    query =  query,
    k = 3,
    filter={"source": "动物"} # 过滤
)

print(f"查询：'{query}' 的结果：")
for i, doc in enumerate(docs):
    print(f"\n结果 {i}")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")

查询：'哺乳动物' 的结果：

结果 0
内容: 猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。
元数据: {'type': '哺乳动物', 'source': '动物'}

结果 1
内容: 狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。
元数据: {'type': '哺乳动物', 'source': '动物'}


In [10]:
docs = db.similarity_search_with_score(
    "量子力学是什么?"
)

for doc, score in docs:
    print(f" [L2距离得分={score:.3f}] {doc.page_content} [{doc.metadata}]")

    # 分数值越小，检索到的文档和问题越相似，取值是[0, 正无穷]

 [L2距离得分=0.182] 量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。 [{'source': '物理', 'type': '科学'}]
 [L2距离得分=0.447] 太阳是太阳系的中心恒星，直径约139万公里，主要由氢和氦组成。它通过核聚变反应产生能量，为地球提供光和热。太阳活动周期约为11年，会影响地球气候。 [{'type': '恒星', 'source': '天文'}]
 [L2距离得分=0.463] 人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。 [{'source': '生物', 'type': '灵长类'}]
 [L2距离得分=0.488] 新冠病毒（SARS-CoV-2）是一种可引起呼吸道疾病的冠状病毒。它通过飞沫传播，主要症状包括发热、咳嗽、乏力。疫苗和戴口罩是有效的预防措施。 [{'type': '病毒', 'source': '医学'}]


In [11]:
docs = db._similarity_search_with_relevance_scores(
    "量子力学是什么？"
)

for doc, score in docs:
    print(f"* [余弦相似度得分={score:.3f}] {doc.page_content} [{doc.metadata}]")

    # 余弦相似度分数搜索，越接近1越相似 cos(0, 180°)

* [余弦相似度得分=0.870] 量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。 [{'source': '物理', 'type': '科学'}]
* [余弦相似度得分=0.681] 太阳是太阳系的中心恒星，直径约139万公里，主要由氢和氦组成。它通过核聚变反应产生能量，为地球提供光和热。太阳活动周期约为11年，会影响地球气候。 [{'source': '天文', 'type': '恒星'}]
* [余弦相似度得分=0.673] 人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。 [{'type': '灵长类', 'source': '生物'}]
* [余弦相似度得分=0.656] 新冠病毒（SARS-CoV-2）是一种可引起呼吸道疾病的冠状病毒。它通过飞沫传播，主要症状包括发热、咳嗽、乏力。疫苗和戴口罩是有效的预防措施。 [{'source': '医学', 'type': '病毒'}]


In [12]:
# MMR 最大边际相关性，max-marginal_relevance_search
docs = db.max_marginal_relevance_search(
    query="量子力学是什么？",
    lambda_mult=0.8     # 侧重相似性（范围 0 - 1）
)

print("关于【量子力学是什么】的搜索结果")
print("=" * 50)

for i, doc in enumerate(docs):
    print(f"\n 结果 {i + 1}")
    print(f"  内容：{doc.page_content}")
    print(f"  标签：{', '.join(f'{k}={v}' for k, v in doc.metadata.items())}")

关于【量子力学是什么】的搜索结果

 结果 1
  内容：量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。
  标签：source=物理, type=科学

 结果 2
  内容：太阳是太阳系的中心恒星，直径约139万公里，主要由氢和氦组成。它通过核聚变反应产生能量，为地球提供光和热。太阳活动周期约为11年，会影响地球气候。
  标签：type=恒星, source=天文

 结果 3
  内容：人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。
  标签：source=生物, type=灵长类

 结果 4
  内容：新冠病毒（SARS-CoV-2）是一种可引起呼吸道疾病的冠状病毒。它通过飞沫传播，主要症状包括发热、咳嗽、乏力。疫苗和戴口罩是有效的预防措施。
  标签：source=医学, type=病毒
